In [8]:
# pip install vnstock pandas numpy

In [2]:
from vnstock import *
import pandas as pd
import numpy as np
import re
import datetime

## Preparation & Listing companies

In [3]:
#Get the list of company tickers
listing_companies = pd.read_csv("/home/jazzdung/Project III/Sample Data/Danh sách công ty/Listing Companies.csv")
listing_companies = listing_companies.sort_values("ticker")
# listing_companies = listing_companies.set_index(['ticker'])
listing_companies['lmao'] = np.arange(1, len(listing_companies) + 1)
listing_companies = listing_companies.set_index(['lmao','ticker'])
listing_companies.to_csv('/home/jazzdung/Project III/Formatted Data/Listing Companies.csv', sep=',', encoding='utf-8')
listing_companies.head()

,,exchange,shortName,industryID,industryIDv2,industry,industryEn,establishedYear,noEmployees,noShareholders,foreignPercent,website,stockRating,deltaInWeek,deltaInMonth,deltaInYear,outstandingShare,issueShare,companyType
lmao,ticker,,,,,,,,,,,,,,,,,,
1,A32,UPCOM,Công ty 32,227.0,3765,Hàng cá nhân & Gia dụng,Personal & Household Goods,2008.0,1098.0,488.0,0.000,http://32jsc.com,NaN,-0.095,0.152,0.354,6.8,6.8,CT
2,AAA,HOSE,An Phát Bioplastics,150.0,1353,Hóa chất,Chemicals,2002.0,2713.0,27697.0,0.025,https://anphatbioplastics.com,2.7,0.000,-0.113,-0.288,382.3,382.3,CT
3,AAM,HOSE,Thủy sản Mekong,217.0,3573,Thực phẩm và đồ uống,Food & Beverage,2002.0,159.0,0.0,0.010,http://www.mekongfish.vn,3.3,-0.001,-0.040,0.120,10.5,12.3,CT
4,AAS,UPCOM,Chứng khoán SmartInvest,300.0,8777,Dịch vụ tài chính,Financial Services,2006.0,43.0,5447.0,0.012,http://www.sisi.com.vn,3.0,-0.095,-0.258,-0.346,80.0,80.0,CK
5,AAT,HOSE,Tập đoàn Tiên Sơn Thanh Hóa,226.0,3763,Hàng cá nhân & Gia dụng,Personal & Household Goods,1995.0,1200.0,2431.0,0.003,http://tiensonaus.com,2.9,-0.171,-0.286,-0.413,63.8,63.8,CT


In [20]:
#Get the list of company tickers
listing_companies = pd.read_csv("/home/jazzdung/Project III/Sample Data/Danh sách công ty/Listing Companies.csv")
# listing_companies.head()
tickers = listing_companies["ticker"].sort_values().values.tolist()
for ticker in tickers:
    if len(ticker) > 3:
        tickers.remove(ticker)

# tickers = tickers[:10]
# print(tickers)

In [31]:
#Convert tickers list to a single string, used as input for some function
inputTicker = ','.join(tickers)

## Income statement

In [13]:
income_statement = financial_flow ('AAA', 'incomestatement' ,'quarterly')
income_statement.reset_index(inplace=True)
income_statement = income_statement.rename(columns = {'index':'switch'})
income_statement[['year','quarter']] = income_statement.switch.str.split("-",expand=True)
income_statement = income_statement.drop('switch', axis=1)
income_statement['quarter'] = income_statement['quarter'].str[1:]
income_statement = income_statement.set_index(['ticker','year','quarter'])
income_statement.drop(income_statement.index, inplace=True)

In [14]:
for ticker in tickers:
    try:
        quarterly_financial_flow = financial_flow (ticker, 'incomestatement' ,'quarterly')
        yearly_financial_flow = financial_flow (ticker, 'incomestatement' ,'yearly')
        concat_financial_flow = pd.concat([quarterly_financial_flow, yearly_financial_flow])

        concat_financial_flow.reset_index(inplace=True)
        concat_financial_flow = concat_financial_flow.rename(columns = {'index':'switch'})
        concat_financial_flow[['year','quarter']] = concat_financial_flow.switch.str.split("-",expand=True)
        concat_financial_flow = concat_financial_flow.drop('switch', axis=1)
        concat_financial_flow['quarter'] = concat_financial_flow['quarter'].str[1:]
        concat_financial_flow = concat_financial_flow.set_index(['ticker','year','quarter'])
        income_statement = pd.concat([income_statement, concat_financial_flow])
    except Exception: 
        pass    

income_statement = income_statement.sort_values(by=['year','quarter','ticker'], ascending=False)

income_statement.to_csv('/home/jazzdung/Project III/Formatted Data/Income Statement.csv', sep=',', encoding='utf-8')

## Balance Sheet

In [15]:
balance_sheet = financial_flow ('AAA', 'balancesheet' ,'quarterly')
balance_sheet.reset_index(inplace=True)
balance_sheet = balance_sheet.rename(columns = {'index':'switch'})
balance_sheet[['year','quarter']] = balance_sheet.switch.str.split("-",expand=True)
balance_sheet = balance_sheet.drop('switch', axis=1)
balance_sheet['quarter'] = balance_sheet['quarter'].str[1:]

balance_sheet = balance_sheet.set_index(['ticker','year','quarter'])
balance_sheet.drop(balance_sheet.index, inplace=True)

In [16]:
for ticker in tickers:
    try:
        quarterly_balance_sheet = financial_flow (ticker, 'balancesheet' ,'quarterly')
        yearly_balance_sheet = financial_flow (ticker, 'balancesheet' ,'yearly')

        single_balance_sheet = pd.concat([quarterly_balance_sheet, yearly_balance_sheet])
        single_balance_sheet.reset_index(inplace=True)
        single_balance_sheet = single_balance_sheet.rename(columns = {'index':'switch'})
        single_balance_sheet[['year','quarter']] = single_balance_sheet.switch.str.split("-",expand=True)
        single_balance_sheet = single_balance_sheet.drop('switch', axis=1)
        single_balance_sheet['quarter'] = single_balance_sheet['quarter'].str[1:]
        single_balance_sheet = single_balance_sheet.set_index(['ticker','year','quarter'])
        balance_sheet = pd.concat([balance_sheet, single_balance_sheet])
    except Exception: 
        pass    

balance_sheet = balance_sheet.sort_values(by=['year','quarter','ticker'], ascending=False)
balance_sheet.to_csv('/home/jazzdung/Project III/Formatted Data/Balance Sheet.csv', sep=',', encoding='utf-8')

## Cash Flow

In [17]:
cash_flow = financial_flow ('AAA', 'cashflow' ,'quarterly')
cash_flow.reset_index(inplace=True)
cash_flow = cash_flow.rename(columns = {'index':'switch'})
cash_flow[['year','quarter']] = cash_flow.switch.str.split("-",expand=True)
cash_flow = cash_flow.drop('switch', axis=1)
cash_flow['quarter'] = cash_flow['quarter'].str[1:]
cash_flow = cash_flow.set_index(['ticker','year','quarter'])
cash_flow.drop(cash_flow.index, inplace=True)

In [18]:
for ticker in tickers:
    try:
        quarterly_cash_flow = financial_flow (ticker, 'cashflow' ,'quarterly')
        yearly_cash_flow = financial_flow (ticker, 'cashflow' ,'yearly')

        single_cash_flow = pd.concat([quarterly_cash_flow, yearly_cash_flow])
        single_cash_flow.reset_index(inplace=True)
        single_cash_flow = single_cash_flow.rename(columns = {'index':'switch'})
        single_cash_flow[['year','quarter']] = single_cash_flow.switch.str.split("-",expand=True)
        single_cash_flow = single_cash_flow.drop('switch', axis=1)
        single_cash_flow['quarter'] = single_cash_flow['quarter'].str[1:]
        single_cash_flow = single_cash_flow.set_index(['ticker','year','quarter'])
        cash_flow = pd.concat([cash_flow, single_cash_flow])
    except Exception: 
        pass    


cash_flow = cash_flow[~cash_flow.index.duplicated(keep='first')]
cash_flow = cash_flow.sort_values(by=['year','quarter','ticker'], ascending=False)
cash_flow.to_csv('/home/jazzdung/Project III/Formatted Data/Cash Flow.csv', sep=',', encoding='utf-8')

## Financial Ratio

In [19]:
all_financial_ratio = financial_ratio('AAA', 'quarterly', True)
all_financial_ratio = all_financial_ratio.set_index(['ticker','year','quarter'])
all_financial_ratio.drop(all_financial_ratio.index, inplace=True)

In [20]:
for ticker in tickers:
    try:
        quarterly_financial_ratio = financial_ratio(ticker, 'quarterly', True)
        yearly_financial_ratio = financial_ratio (ticker, 'yearly', True)

        single_financial_ratio = pd.concat([quarterly_financial_ratio, yearly_financial_ratio])
        single_financial_ratio = single_financial_ratio.set_index(['ticker','year','quarter'])
        all_financial_ratio = pd.concat([all_financial_ratio, single_financial_ratio])
    except Exception: 
        pass    


all_financial_ratio = all_financial_ratio.sort_values(by=['year','quarter','ticker'], ascending=False)
all_financial_ratio.to_csv('/home/jazzdung/Project III/Formatted Data/Financial Ratio.csv', sep=',', encoding='utf-8')

## Stock History

In [21]:
all_stock =  stock_historical_data(symbol='GMD', start_date="2022-10-01", end_date='2022-11-22')
all_stock['ticker'] = 'GMD'

#Get time stamp '2004-10-19 10:23:54'
all_stock['timeStamp'] = all_stock['TradingDate'].astype(str) + " 00:00:00"
all_stock = all_stock.drop(['TradingDate'], axis =1)
all_stock = all_stock.set_index(['ticker','timeStamp'])
# all_stock.head()
all_stock.drop(all_stock.index, inplace=True)

In [22]:
for ticker in tickers:
    try:
        single_stock =  stock_historical_data(symbol=ticker, start_date="2021-01-01", end_date='2022-11-22')
        single_stock['ticker'] = ticker
        single_stock['timeStamp'] = single_stock['TradingDate'].astype(str) + " 00:00:00"
        single_stock = single_stock.drop(['TradingDate'], axis =1)
        single_stock = single_stock.drop_duplicates( subset = ['ticker', 'timeStamp'], keep = 'first').reset_index(drop = True)
        single_stock = single_stock.set_index(['ticker','timeStamp'])
        
        all_stock = pd.concat([all_stock, single_stock])
    except Exception: 
        pass    



all_stock.to_csv('/home/jazzdung/Project III/Formatted Data/Stock History.csv', sep=',', encoding='utf-8')

## Stock Intraday Data

- Runable after 9AM (Stock market opening) 

In [29]:
today_stock_data = stock_intraday_data(symbol='ACB', page_num=0, page_size=5000)
today_stock_data['timeStamp'] = datetime.datetime.today().strftime('%Y-%m-%d') + " " + today_stock_data['time'].astype(str)
today_stock_data['ticker'] = 'ACB'
today_stock_data = today_stock_data.drop(['time'], axis =1 )
today_stock_data.drop(today_stock_data.index, inplace=True)
today_stock_data.head()

,price,volume,cp,rcp,a,ba,sa,hl,pcp,timeStamp,ticker


In [ ]:
for ticker in tickers:
    try:
        single_stock_data =  stock_intraday_data(symbol=ticker, page_num=0, page_size=5000)
        single_stock_data['timeStamp'] = datetime.datetime.today().strftime('%Y-%m-%d') + " " + single_stock_data['time'].astype(str)
        single_stock_data['ticker'] = ticker
        single_stock_data = single_stock_data.drop(['time'], axis =1 )
        today_stock_data = pd.concat([today_stock_data, single_stock_data])
    except Exception: 
        pass    


today_stock_data = today_stock_data.reset_index(drop=True)
today_stock_data.index = range(0, len(today_stock_data))
# today_stock_data = today_stock_data.drop(['level_0'], axis =1)
# today_stock_data.head()
today_stock_data.to_csv('/home/jazzdung/Project III/Formatted Data/Stock Intraday Transaction.csv', sep=',', encoding='utf-8')

## Price board

In [ ]:
inputTicker = ','.join(tickers)
currentPriceBoard = price_board(inputTicker)
currentPriceBoard['timeStamp'] = str(datetime.datetime.now())[:-7]


currentPriceBoard.to_csv('/home/jazzdung/Project III/Formatted Data/Price Board.csv', sep=',', encoding='utf-8')

## Stock rating

- General rating

In [ ]:
all_stock_general_rating = general_rating('AAA')
all_stock_general_rating['ticker'] = 'AAA'
all_stock_general_rating = all_stock_general_rating.set_index(['ticker'])
all_stock_general_rating.drop(all_stock_general_rating.index, inplace=True)
# all_stock_general_rating.head()

for ticker in tickers:
    try:
        single_stock_general_rating =  general_rating(ticker)
        single_stock_general_rating['ticker'] = ticker
        single_stock_general_rating = single_stock_general_rating.set_index(['ticker'])
        
        all_stock_general_rating = pd.concat([all_stock_general_rating, single_stock_general_rating])
    except Exception: 
        pass    


all_stock_general_rating.to_csv('/home/jazzdung/Project III/Formatted Data/General Rating.csv', sep=',', encoding='utf-8')

- Business model rating

In [26]:
all_stock_business_model_rating = biz_model_rating('AAA')
all_stock_business_model_rating['ticker'] = 'AAA'
all_stock_business_model_rating = all_stock_business_model_rating.set_index(['ticker'])
all_stock_business_model_rating.drop(all_stock_business_model_rating.index, inplace=True)
# all_stock_business_model_rating.head()

for ticker in tickers:
    try:
        single_stock_business_model_rating =  biz_model_rating(ticker)
        single_stock_business_model_rating['ticker'] = ticker
        single_stock_business_model_rating = single_stock_business_model_rating.set_index(['ticker'])
        
        all_stock_business_model_rating = pd.concat([all_stock_business_model_rating, single_stock_business_model_rating])
    except Exception: 
        pass    



all_stock_business_model_rating.to_csv('/home/jazzdung/Project III/Formatted Data/Business Model Rating.csv', sep=',', encoding='utf-8')

- Business operation rating

In [27]:
all_stock_business_operation_rating = biz_operation_rating('AAA')
all_stock_business_operation_rating['ticker'] = 'AAA'
all_stock_business_operation_rating = all_stock_business_operation_rating.set_index(['ticker'])
all_stock_business_operation_rating.drop(all_stock_business_operation_rating.index, inplace=True)
# all_stock_business_operation_rating.head()

for ticker in tickers:
    try:
        single_stock_business_operation_rating =  biz_operation_rating(ticker)
        single_stock_business_operation_rating['ticker'] = ticker
        single_stock_business_operation_rating = single_stock_business_operation_rating.set_index(['ticker'])
        
        all_stock_business_operation_rating = pd.concat([all_stock_business_operation_rating, single_stock_business_operation_rating])
    except Exception: 
        pass    


all_stock_business_operation_rating.to_csv('/home/jazzdung/Project III/Formatted Data/Business Operation Rating.csv', sep=',', encoding='utf-8')

- Financial health rating

In [28]:
all_stock_financial_health_rating = financial_health_rating('AAA')
all_stock_financial_health_rating['ticker'] = 'AAA'
all_stock_financial_health_rating = all_stock_financial_health_rating.set_index(['ticker'])
all_stock_financial_health_rating.drop(all_stock_financial_health_rating.index, inplace=True)
# all_stock_financial_health_rating.head()

for ticker in tickers:
    try:
        single_stock_financial_health_rating =  financial_health_rating(ticker)
        single_stock_financial_health_rating['ticker'] = ticker
        single_stock_financial_health_rating = single_stock_financial_health_rating.set_index(['ticker'])
        
        all_stock_financial_health_rating = pd.concat([all_stock_financial_health_rating, single_stock_financial_health_rating])
    except Exception: 
        pass    


all_stock_financial_health_rating.to_csv('/home/jazzdung/Project III/Formatted Data/Financial Health Rating.csv', sep=',', encoding='utf-8')

- Valuation rating

In [29]:
all_stock_valuation_rating = valuation_rating('AAA')
all_stock_valuation_rating['ticker'] = 'AAA'
all_stock_valuation_rating = all_stock_valuation_rating.set_index(['ticker'])
all_stock_valuation_rating.drop(all_stock_valuation_rating.index, inplace=True)
# all_stock_valuation_rating.head()

for ticker in tickers:
    try:
        single_stock_valuation_rating =  valuation_rating(ticker)
        single_stock_valuation_rating['ticker'] = ticker
        single_stock_valuation_rating = single_stock_valuation_rating.set_index(['ticker'])
        
        all_stock_valuation_rating = pd.concat([all_stock_valuation_rating, single_stock_valuation_rating])
    except Exception: 
        pass    


all_stock_valuation_rating.to_csv('/home/jazzdung/Project III/Formatted Data/Valuation Rating.csv', sep=',', encoding='utf-8')

- Industry financial rating

In [30]:
all_stock_industry_financial_health = industry_financial_health('AAA')
all_stock_industry_financial_health['ticker'] = 'AAA'
all_stock_industry_financial_health = all_stock_industry_financial_health.set_index(['ticker'])
all_stock_industry_financial_health.drop(all_stock_industry_financial_health.index, inplace=True)
# all_stock_industry_financial_health.head()

for ticker in tickers:
    try:
        single_stock_industry_financial_health =  industry_financial_health(ticker)
        single_stock_industry_financial_health['ticker'] = ticker
        single_stock_industry_financial_health = single_stock_industry_financial_health.set_index(['ticker'])
        
        all_stock_industry_financial_health = pd.concat([all_stock_industry_financial_health, single_stock_industry_financial_health])
    except Exception: 
        pass    


all_stock_industry_financial_health.to_csv('/home/jazzdung/Project III/Formatted Data/Industry Financial Health.csv', sep=',', encoding='utf-8')

- Industry analysis

In [25]:
all_stock_industry_financial_health = industry_analysis(inputTicker)
all_stock_industry_financial_health.head()

,ticker,marcap,price,numberOfDays,priceToEarning,peg,priceToBook,valueBeforeEbitda,dividend,roe,...,debtOnEquity,debtOnEbitda,income5year,sale5year,income1quarter,sale1quarter,nextIncome,nextSale,rsi,rs
0,BII,138,2400,0,-2.5,0.0,0.3,-11.1,0.0,-0.097,...,0.2,-5.5,0.35,0.286,None,2.146,None,None,52.7,5.0


- Stock list analysis

In [24]:
all_stock_industry_financial_health = stock_ls_analysis(inputTicker)
all_stock_industry_financial_health.head()

,ticker,marcap,numberOfDays,dividend,roe,roa,currentPayment,quickPayment,grossProfitMargin,debtOnEquity,debtOnEbitda,rsi,rs
0,BBC,1121,-2,0.016,0.144,0.104,1.3,1.0,0.325,0.2,1.3,72.1,62.0
1,SCR,2346,1,0.000,0.030,0.014,1.9,1.0,0.372,0.4,30.6,51.8,33.0
2,SCS,6891,1,0.000,0.491,0.442,7.1,7.1,0.804,0.0,0.0,60.4,68.0
3,SCY,589,0,0.000,0.046,0.027,1.6,0.8,0.086,0.0,0.0,46.0,34.0
4,SD2,100,0,0.000,0.012,0.004,1.6,1.0,0.055,0.5,15.3,70.5,42.0


## Create table script generator

In [34]:
def formatNames(dataFrame):
    fields = list(dataFrame.columns.values)

    for i in range(len(fields)):
        fields[i] = fields[i][0].lower() + fields[i][1:]
        fields[i] = re.sub(r"(\w)([A-Z])", r"\1_\2", fields[i])
        fields[i] = fields[i].lower()
    
    if fields[0] == "unnamed: 0":
        fields[0] = "index"
    return(fields)

def getDataType(dataFrame):
    oldDataType = dataFrame.dtypes
    newDataType = []
    for i in range(len(oldDataType)):
        if oldDataType[i] == "float64":
            newDataType.append("double precision")  
        elif oldDataType[i] == "int64":
            newDataType.append("integer")
        elif oldDataType[i] == "object":
            maxDataSize = int(dataFrame.iloc[:,i].str.len().max())
            if maxDataSize < 20:
                newDataType.append("varchar(" + str(maxDataSize) + ")")
            else:
                newDataType.append("varchar(256)")
        else:
            newDataType.append("varchar(256)")

    return newDataType

def tableScript(dataFrame, tableName, primaryKeyIndex):
    fields = formatNames(dataFrame)
    dataTypes = getDataType(dataFrame)

    #Create table query
    createTable = "CREATE TABLE IF NOT EXISTS " + tableName + " (" 

    for i in range(len(fields)):    
        createField = "\n \t" + fields[i] + " " + dataTypes[i] + "," 
        createTable += createField

    createTable += "\n \t" + "PRIMARY KEY (" 

    for index in primaryKeyIndex:
        createTable += fields[index] + ", "

    createTable = createTable[:-2]
    createTable += "),"
    createTable += "\n \t" + "FOREIGN KEY (ticker) REFERENCES listing_companies (ticker)"
    createTable += "\n );"

    return createTable

In [35]:
stock_history_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Stock History.csv")
income_statement_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Income Statement.csv")
balance_sheet_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Balance Sheet.csv")
cash_flow_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Cash Flow.csv")
listing_companies_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Listing Companies.csv")
financial_ratio_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Financial Ratio.csv")
stock_intraday_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Stock Intraday Transaction.csv")
price_board_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Price Board.csv")

general_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/General Rating.csv")
business_model_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Business Model Rating.csv")
business_operation_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Business Operation Rating.csv")
financial_health_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Financial Health Rating.csv")
valuation_rating_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Valuation Rating.csv")
industry_financial_health_table = pd.read_csv("/home/jazzdung/Project III/Formatted Data/Industry Financial Health.csv")


bigAssScript = ""

bigAssScript += tableScript(listing_companies_table, "listing_companies", [0]) + "\n \n \n"
bigAssScript += tableScript(stock_history_table, "stock_history", [0,1]) + "\n \n \n"
bigAssScript += tableScript(income_statement_table, "income_statement", [0,1,2]) + "\n \n \n"
bigAssScript += tableScript(balance_sheet_table, "balance_sheet", [0,1,2]) + "\n \n \n"
bigAssScript += tableScript(cash_flow_table, "cash_flow", [0,1,2]) + "\n \n \n"
bigAssScript += tableScript(financial_ratio_table, "financial_ratio", [0,1,2]) + "\n \n \n"


bigAssScript += tableScript(general_rating_table, "general_rating", [0]) + "\n \n \n"
bigAssScript += tableScript(business_model_rating_table, "business_model_rating", [0]) + "\n \n \n"
bigAssScript += tableScript(business_operation_rating_table, "business_operation_rating", [0]) + "\n \n \n"
bigAssScript += tableScript(financial_health_rating_table, "financial_health_rating", [0]) + "\n \n \n"
bigAssScript += tableScript(valuation_rating_table, "valuation_rating", [0]) + "\n \n \n"
bigAssScript += tableScript(industry_financial_health_table, "industry_financial_health", [0]) + "\n \n \n"


bigAssScript += tableScript(stock_intraday_table, "stock_intraday_transaction", [0,1,2]) + "\n \n \n"
# bigAssScript += tableScript(price_board_table, "price_board", [0,1,2]) + "\n \n \n"
bigAssScript.replace("¶", "")

sqlFile = open("/home/jazzdung/Project III/SQL Script/Create Table (Template).sql", "w") 
sqlFile.write(bigAssScript)
sqlFile.close()

In [25]:
# print(bigAssScript)